In [1]:
from forcefield import CGForceField
from model_builder import CGModelBuilder

In [5]:
import openmm.app as app

builder = CGModelBuilder(
    smiles_sequence="[OH][EO]10[OH]",
    bond_length=0.35,  # nm
    min_dist=0.35,  # nm
    max_tries=50000,
)
topology, positions = builder.create_molecule()

with open("peo10.pdb", "w") as f:
    app.PDBFile.writeFile(topology, positions, f)


In [6]:
structures = [
    {
        "pdb": "peo10.pdb",
        "number": 10,
        "box": (0, 0, 0, 50, 50, 50),  # Aを10個  50Å立方
    },
]

topology, positions = builder.create_packed_model(
    structures,
    output_pdb="peo10_10.pdb",  # 生成されるPACKMOL出力
)
pdb = app.PDBFile("peo10_10.pdb")
topology = pdb.topology
positions = pdb.positions

In [7]:
param_dict = {
    "BOND": {
        ("EO", "EO"): {"r0": 0.5, "k": 100},
        ("EO", "OH"): {"r0": 0.45, "k": 120},
    },
    "NONBONDED": {
        "EO": {"sigma": 0.3, "epsilon": 0.5, "charge": 0.0, "mass": 72.0},
        "OH": {"sigma": 0.25, "epsilon": 0.4, "charge": -0.2, "mass": 18.0},
    },
    "ANGLE": {},
    "TORSION": {},
}
ff = CGForceField.from_dict(param_dict)
system = ff.create_system(topology)

In [9]:
import sys

import openmm
from openmm import unit

temperature = 300 * unit.kelvin
friction = 1 / unit.picosecond
timestep = 0.001 * unit.picoseconds
integrator = openmm.LangevinIntegrator(temperature, friction, timestep)

simulation = app.Simulation(topology, system, integrator)
simulation.context.setPositions(positions)
simulation.reporters.append(app.PDBReporter("example-traj-packmol.pdb", 1000))
simulation.reporters.append(
    app.StateDataReporter(
        sys.stdout,
        1000,
        step=True,
        potentialEnergy=True,
        kineticEnergy=True,
        temperature=True,
        volume=True,
        density=True,
    )
)
simulation.minimizeEnergy()
simulation.step(10_000)


#"Step","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Density (g/mL)"
1000,457.3514276900267,344.3541717598373,230.09048447968348,8.0,1.5692094184792849
2000,544.7623669165666,448.08771780467066,299.4031393671566,8.0,1.5692094184792849
3000,550.4264583489502,424.383750239875,283.5646282400768,8.0,1.5692094184792849
4000,495.90025057760454,496.7150380690085,331.89493007609434,8.0,1.5692094184792849
5000,504.50962182601717,446.2988180326522,298.20783276428966,8.0,1.5692094184792849
6000,482.8565235319272,459.2865090287836,306.88594489024484,8.0,1.5692094184792849
7000,489.42599646196555,410.0474613117083,273.9854102847236,8.0,1.5692094184792849
8000,421.15480908583595,499.7672759074356,333.9343735926396,8.0,1.5692094184792849
9000,461.4152781894957,430.10803722431547,287.38948088762726,8.0,1.5692094184792849
10000,497.9938395128056,445.358872077522,297.57977991075416,8.0,1.5692094184792849


In [25]:
import MDAnalysis as mda
import nglview as nv

bonds = list(topology.bonds())
bonds_nglview = []
for bond in bonds:
    bonds_nglview.append((bond[0].index, bond[1].index))
u = mda.Universe("example-traj-packmol.pdb")
u.add_TopologyAttr("bonds", bonds_nglview)
view = nv.show_mdanalysis(u)
view.add_ball_and_stick()
view.add_representation("ball+stick", colorScheme="resname")
view

NGLWidget(max_frame=9)